In [1]:
import sys
sys.path.append("../")

from zhipuai_embedding import ZhipuAIEmbeddings

from langchain.vectorstores.chroma import Chroma
from langchain_openai import ChatOpenAI
import os

zhipuai_api_key = os.environ['ZHIPUAI_API_KEY']
OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]

embedding = ZhipuAIEmbeddings()

persist_directory = "../data_base/vector_db/chroma"

vectordb = Chroma(
    persist_directory=persist_directory,
    embedding_function=embedding
)

llm = ChatOpenAI(model_name="gpt-3.5-turbo",temperature=0,openai_api_key=os.environ['OPENAI_API_KEY'],openai_api_base=os.environ["BASE_URL"])


In [2]:
#人工评估的一般思路

#准则一 量化评估

#A Prompt 简明扼要
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA


template_v1 = """使用以下上下文来回答最后的问题。如果你不知道答案，就说你不知道，不要试图编造答
案。最多使用三句话。尽量使答案简明扼要。总是在回答的最后说“谢谢你的提问！”。
{context}
问题: {question}
"""

QA_CHAIN_PROMPT = PromptTemplate(input_variables=["context","question"],
                                 template=template_v1)



qa_chain = RetrievalQA.from_chain_type(llm,
                                       retriever=vectordb.as_retriever(),
                                       return_source_documents=True,
                                       chain_type_kwargs={"prompt":QA_CHAIN_PROMPT})

print("问题一：")
question = "南瓜书和西瓜书有什么关系？"
result = qa_chain({"query": question})
print(result["result"])

print("问题二：")
question = "应该如何使用南瓜书？"
result = qa_chain({"query": question})
print(result["result"])


问题一：


/tmp/ipykernel_15878/327751208.py:27: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa_chain({"query": question})


南瓜书是对西瓜书的解析和补充公式推导细节的一本书。南瓜书建议以西瓜书为主线，在遇到推导困难时再查阅南瓜书。谢谢你的提问！
问题二：
应该以《机器学习》（西瓜书）为主线，遇到推导不出来或者看不懂的公式时再来查阅南瓜书。对于初学者不建议深究第1章和第2章的公式，等学得有点基础再回来。若需要补充或查阅新公式，可以在GitHub上反馈或联系作者。谢谢你的提问！


In [4]:
#B Prompt 详细具体
template_v2 = """使用以下上下文来回答最后的问题。如果你不知道答案，就说你不知道，不要试图编造答
案。你应该使答案尽可能详细具体，但不要偏题。如果答案比较长，请酌情进行分段，以提高答案的阅读体验。
{context}
问题: {question}
有用的回答:"""

QA_CHAIN_PROMPT = PromptTemplate(input_variables=["context","question"],
                                 template=template_v2)

qa_chain = RetrievalQA.from_chain_type(llm,
                                       retriever=vectordb.as_retriever(),
                                       return_source_documents=True,
                                       chain_type_kwargs={"prompt":QA_CHAIN_PROMPT})

print("问题一：")
question = "南瓜书和西瓜书有什么关系？"
result = qa_chain({"query": question})
print(result["result"])

print("问题二：")
question = "应该如何使用南瓜书？"
result = qa_chain({"query": question})
print(result["result"])


问题一：
南瓜书是以西瓜书《机器学习》为前置知识进行表述的补充性书籍，旨在解析西瓜书中比较难理解的公式并补充具体的推导细节。西瓜书是机器学习领域的经典入门教材之一，而南瓜书则是对西瓜书的补充和解析。南瓜书的最佳使用方法是以西瓜书为主线，当遇到自己推导不出来或者看不懂的公式时再来查阅南瓜书。因此，可以说南瓜书是在西瓜书基础上的进一步解析和补充，帮助读者更好地理解机器学习领域的知识和公式。
问题二：
使用南瓜书的最佳方法是以周志华老师的《机器学习》（西瓜书）作为主线，先阅读西瓜书内容，遇到推导不出来或看不懂的公式时再来查阅南瓜书。西瓜书是机器学习领域的经典入门教材，南瓜书则旨在对西瓜书中一些难以理解的公式进行解析和补充推导细节，帮助读者更好地理解和掌握机器学习知识。对于初学机器学习的小白，建议先简单过一下南瓜书第1章和第2章的公式，等学有一定基础后再深入研究。南瓜书的编写力争用本科数学基础的视角进行讲解，超纲的数学知识会在附录和参考文献中给出，感兴趣的同学可以继续深入学习。若在南瓜书中未找到想要查阅的公式或发现错误，请通过GitHub进行反馈。南瓜书还配有相关视频教程和在线阅读地址，欢迎学习交流。


In [5]:
#准则二 多维评估

print("问题：")
question = "应该如何使用南瓜书？"
print(question)
print("模型回答：")
result = qa_chain({"query": question})
print(result["result"])

问题：
应该如何使用南瓜书？
模型回答：
要使用南瓜书，最好以西瓜书为主线，先进行整体的学习和理解。当遇到自己无法推导或理解的公式时，可以查阅南瓜书进行详细解析和推导。对于初学机器学习的小白来说，建议先简单浏览南瓜书第1章和第2章的公式，等学习逐渐深入后再回来深入理解公式。每个公式的解析和推导都力求以本科数学基础的视角进行讲解，超纲的数学知识会以附录和参考文献的形式给出。如果需要查找特定公式或提交勘误信息，可以前往南瓜书的GitHub页面提交反馈。此外，南瓜书还有配套的视频教程和在线阅读地址供参考。最终，希望通过南瓜书的学习能够帮助大家成为合格的机器学习从业者。


In [6]:
print(result["source_documents"])

[Document(metadata={'author': '', 'creationDate': "D:20230303170709-00'00'", 'creator': 'LaTeX with hyperref', 'file_path': '../data_base/knowledge_db/pumkin_book/pumpkin_book.pdf', 'format': 'PDF 1.5', 'keywords': '', 'modDate': '', 'page': 1, 'producer': 'xdvipdfmx (20200315)', 'source': '../data_base/knowledge_db/pumkin_book/pumpkin_book.pdf', 'subject': '', 'title': '', 'total_pages': 196, 'trapped': ''}, page_content='为主线，遇到自己推导不出来或者看不懂的公式时再来查阅南瓜书；\n• 对于初学机器学习的小白，西瓜书第1 章和第2 章的公式强烈不建议深究，简单过一下即可，等你学得\n有点飘的时候再回来啃都来得及；\n• 每个公式的解析和推导我们都力(zhi) 争(neng) 以本科数学基础的视角进行讲解，所以超纲的数学知识\n我们通常都会以附录和参考文献的形式给出，感兴趣的同学可以继续沿着我们给的资料进行深入学习；\n• 若南瓜书里没有你想要查阅的公式，\n或者你发现南瓜书哪个地方有错误，\n请毫不犹豫地去我们GitHub 的\nIssues（地址：https://github.com/datawhalechina/pumpkin-book/issues）进行反馈，在对应版块\n提交你希望补充的公式编号或者勘误信息，我们通常会在24 小时以内给您回复，超过24 小时未回复的\n话可以微信联系我们（微信号：at-Sm1les）\n；\n配套视频教程：https://www.bilibili.com/video/BV1Mh411e7VU'), Document(metadata={'author': '', 'creationDate': "D:20230303170709-00'00'", 'creator': 'LaTeX with hyper

In [7]:
#简单自动评估
#方法一 构造客观题

prompt_template = '''
请你做如下选择题：
题目：南瓜书的作者是谁？
选项：A 周志明 B 谢文睿 C 秦州 D 贾彬彬
你可以参考的知识片段：
~~~
{}
~~~
请仅返回选择的选项
如果你无法做出选择，请返回空
'''

In [8]:
#多选题评分函数 全对1分，漏选0.5分，错选不得分
def multi_select_score_v1(true_answer : str, generate_answer : str) -> float:
    # true_anser : 正确答案，str 类型，例如 'BCD'
    # generate_answer : 模型生成答案，str 类型
    true_answers = list(true_answer)
    '''为便于计算，我们假设每道题都只有 A B C D 四个选项'''
    # 先找出错误答案集合
    false_answers = [item for item in ['A', 'B', 'C', 'D'] if item not in true_answers]
    # 如果生成答案出现了错误答案
    for one_answer in false_answers:
        if one_answer in generate_answer:
            return 0
    # 再判断是否全选了正确答案
    if_correct = 0
    for one_answer in true_answers:
        if one_answer in generate_answer:
            if_correct += 1
            continue
    if if_correct == 0:
        # 不选
        return 0
    elif if_correct == len(true_answers):
        # 全选
        return 1
    else:
        # 漏选
        return 0.5


In [9]:
answer1 = 'B C'
answer2 = '西瓜书的作者是 A 周志华'
answer3 = '应该选择 B C D'
answer4 = '我不知道'
true_answer = 'BCD'
print("答案一得分：", multi_select_score_v1(true_answer, answer1))
print("答案二得分：", multi_select_score_v1(true_answer, answer2))
print("答案三得分：", multi_select_score_v1(true_answer, answer3))
print("答案四得分：", multi_select_score_v1(true_answer, answer4))

答案一得分： 0.5
答案二得分： 0
答案三得分： 1
答案四得分： 0


In [10]:
#规则变更 错选扣一分
def multi_select_score_v2(true_answer : str, generate_answer : str) -> float:
    # true_anser : 正确答案，str 类型，例如 'BCD'
    # generate_answer : 模型生成答案，str 类型
    true_answers = list(true_answer)
    '''为便于计算，我们假设每道题都只有 A B C D 四个选项'''
    # 先找出错误答案集合
    false_answers = [item for item in ['A', 'B', 'C', 'D'] if item not in true_answers]
    # 如果生成答案出现了错误答案
    for one_answer in false_answers:
        if one_answer in generate_answer:
            return -1
    # 再判断是否全选了正确答案
    if_correct = 0
    for one_answer in true_answers:
        if one_answer in generate_answer:
            if_correct += 1
            continue
    if if_correct == 0:
        # 不选
        return 0
    elif if_correct == len(true_answers):
        # 全选
        return 1
    else:
        # 漏选
        return 0.5


In [11]:
answer1 = 'B C'
answer2 = '西瓜书的作者是 A 周志华'
answer3 = '应该选择 B C D'
answer4 = '我不知道'
true_answer = 'BCD'
print("答案一得分：", multi_select_score_v2(true_answer, answer1))
print("答案二得分：", multi_select_score_v2(true_answer, answer2))
print("答案三得分：", multi_select_score_v2(true_answer, answer3))
print("答案四得分：", multi_select_score_v2(true_answer, answer4))

答案一得分： 0.5
答案二得分： -1
答案三得分： 1
答案四得分： 0


In [13]:
# 实际测试
question = """
请你做如下选择题：
题目：南瓜书的作者是谁？
选项：A 周志明 B 谢文睿 C 秦州 D 贾彬彬
你可以参考的知识片段：
请仅返回选择的选项
如果你无法做出选择，请返回空
"""
result = qa_chain({"query": question})
print(f"答案是:{result['result']}")
print(f"答案得分：{multi_select_score_v2(true_answer,result['result'])}")

答案是:A 周志明
答案得分：-1


In [15]:
#方法二：计算答案相似度
from nltk.translate.bleu_score import sentence_bleu
import jieba

def bleu_score(true_answer : str, generate_answer : str) -> float:
    # true_anser : 标准答案，str 类型
    # generate_answer : 模型生成答案，str 类型
    true_answers = list(jieba.cut(true_answer))
    # print(true_answers)
    generate_answers = list(jieba.cut(generate_answer))
    # print(generate_answers)
    bleu_score = sentence_bleu(true_answers, generate_answers)
    return bleu_score

In [17]:
true_answer = '周志华老师的《机器学习》（西瓜书）是机器学习领域的经典入门教材之一，周老师为了使尽可能多的读者通过西瓜书对机器学习有所了解, 所以在书中对部分公式的推导细节没有详述，但是这对那些想深究公式推导细节的读者来说可能“不太友好”，本书旨在对西瓜书里比较难理解的公式加以解析，以及对部分公式补充具体的推导细节。'

print("答案一：")
answer1 = '周志华老师的《机器学习》（西瓜书）是机器学习领域的经典入门教材之一，周老师为了使尽可能多的读者通过西瓜书对机器学习有所了解, 所以在书中对部分公式的推导细节没有详述，但是这对那些想深究公式推导细节的读者来说可能“不太友好”，本书旨在对西瓜书里比较难理解的公式加以解析，以及对部分公式补充具体的推导细节。'
print(answer1)
score = bleu_score(true_answer, answer1)
print("得分：", score)
print("答案二：")
answer2 = '本南瓜书只能算是我等数学渣渣在自学的时候记下来的笔记，希望能够帮助大家都成为一名合格的“理工科数学基础扎实点的大二下学生”'
print(answer2)
score = bleu_score(true_answer, answer2)
print("得分：", score)

答案一：
周志华老师的《机器学习》（西瓜书）是机器学习领域的经典入门教材之一，周老师为了使尽可能多的读者通过西瓜书对机器学习有所了解, 所以在书中对部分公式的推导细节没有详述，但是这对那些想深究公式推导细节的读者来说可能“不太友好”，本书旨在对西瓜书里比较难理解的公式加以解析，以及对部分公式补充具体的推导细节。
得分： 1.2705543769116016e-231
答案二：
本南瓜书只能算是我等数学渣渣在自学的时候记下来的笔记，希望能够帮助大家都成为一名合格的“理工科数学基础扎实点的大二下学生”
得分： 1.1935398790363042e-231


In [21]:
# 实际测试
question = "南瓜书的目标是什么？"
result = qa_chain({"query": question})
print(f"答案是:{result['result']}")
print(f"答案得分：{bleu_score(true_answer,result['result'])}")

答案是:南瓜书的目标是解析和补充《机器学习》（西瓜书）中比较难理解的公式，为那些想深究公式推导细节的读者提供帮助。其内容以西瓜书为前置知识，通过对公式的解析和补充推导细节，希望帮助读者更好地理解和掌握机器学习的相关知识，使他们成为合格的学习者。
答案得分：1.2423111676412644e-231


/root/miniconda3/envs/llm-universe/lib/python3.10/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/root/miniconda3/envs/llm-universe/lib/python3.10/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/root/miniconda3/envs/llm-universe/lib/python3.10/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consid

In [23]:
#使用大模型进行评估

prompt = '''
你是一个模型回答评估员。
接下来，我将给你一个问题、对应的知识片段以及模型根据知识片段对问题的回答。
请你依次评估以下维度模型回答的表现，分别给出打分：

① 知识查找正确性。评估系统给定的知识片段是否能够对问题做出回答。如果知识片段不能做出回答，打分为0；如果知识片段可以做出回答，打分为1。

② 回答一致性。评估系统的回答是否针对用户问题展开，是否有偏题、错误理解题意的情况，打分分值在0~1之间，0为完全偏题，1为完全切题。

③ 回答幻觉比例。该维度需要综合系统回答与查找到的知识片段，评估系统的回答是否出现幻觉，打分分值在0~1之间,0为全部是模型幻觉，1为没有任何幻觉。

④ 回答正确性。该维度评估系统回答是否正确，是否充分解答了用户问题，打分分值在0~1之间，0为完全不正确，1为完全正确。

⑤ 逻辑性。该维度评估系统回答是否逻辑连贯，是否出现前后冲突、逻辑混乱的情况。打分分值在0~1之间，0为逻辑完全混乱，1为完全没有逻辑问题。

⑥ 通顺性。该维度评估系统回答是否通顺、合乎语法。打分分值在0~1之间，0为语句完全不通顺，1为语句完全通顺没有任何语法问题。

⑦ 智能性。该维度评估系统回答是否拟人化、智能化，是否能充分让用户混淆人工回答与智能回答。打分分值在0~1之间，0为非常明显的模型回答，1为与人工回答高度一致。

你应该是比较严苛的评估员，很少给出满分的高评估。
用户问题：
~~~
{}
~~~
待评估的回答：
~~~
{}
~~~
给定的知识片段：
~~~
{}
~~~
你应该返回给我一个可直接解析的 Python 字典，字典的键是如上维度，值是每一个维度对应的评估打分。
不要输出任何其他内容。
'''

In [25]:
from openai import OpenAI
client = OpenAI(
    api_key=os.environ['OPENAI_API_KEY'],
    base_url=os.environ["BASE_URL"]
)

def gen_gpt_messages(prompt):
    messages = [{"role": "user", "content": prompt}]
    return messages

def get_completion(prompt, model="gpt-3.5-turbo", temperature = 0):
    response = client.chat.completions.create(
        model=model,
        messages=gen_gpt_messages(prompt),
        temperature=temperature,
    )
    if len(response.choices) > 0:
        return response.choices[0].message.content
    return "generate answer error"

question = "应该如何使用南瓜书？"
result = qa_chain({"query":question})
answer = result['result']
knowledge = result["source_documents"]

response = get_completion(prompt.format(question,answer,knowledge))
print(response)

{
    "知识查找正确性": 1,
    "回答一致性": 0.8,
    "回答幻觉比例": 0.9,
    "回答正确性": 0.8,
    "逻辑性": 0.9,
    "通顺性": 0.9,
    "智能性": 0.6
}
